In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau
from scipy.stats import shapiro
from scipy.stats import anderson
from scipy.stats import kstest
from scipy.stats import normaltest
from IPython.display import display
import pandas as pd
from scipy.stats import pearsonr, spearmanr, kendalltau
from typing import List, Union, Dict
import math


''' Notebook nettoyage '''

def analyze_missing_data(df):
    """
    Cette fonction analyse les données manquantes dans un DataFrame.
    
    Paramètre:
    df (pd.DataFrame): Le DataFrame à analyser.

    Affiche le nombre total de cellules manquantes et leur pourcentage.
    """
    # Calcul du nombre total de cellules manquantes
    total_missing = df.isnull().sum().sum()

    # Calcul du nombre total de cellules
    total_cells = df.size

    # Calcul du pourcentage de cellules manquantes
    missing_percentage = (total_missing / total_cells) * 100

    # Affichage des résultats
    print(f"Nombre total de cellules manquantes : {total_missing}")
    print(f"Nombre de cellules manquantes en % : {missing_percentage:.2f}%")


def generate_missingno_bar_graph(df):
    """
    Génère un graphique en barres des valeurs manquantes dans un DataFrame en utilisant missingno.
    
    Args:
    dataframe (pd.DataFrame): Le DataFrame pour lequel générer le graphique.
    
    Returns:
    None: Affiche le graphique.
    """
    # Générer le graphique en barres des valeurs manquantes
    msno.bar(df, color='dodgerblue', fontsize=12, labels=True)
    
    # Customiser le graphique pour être plus moderne
    plt.title('Missing Values Bar Graph', fontsize=16)
    plt.xlabel('Columns', fontsize=14)
    plt.ylabel('Missing Values Count', fontsize=14)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Afficher le graphique
    plt.show()


def missing_cells(df):
    '''Calcule le nombre de cellules manquantes sur le data set total.
    Keyword arguments:
    df -- le dataframe

    return : le nombre de cellules manquantes de df
    '''
    return df.isna().sum().sum()


def missing_cells_perc(df):
    '''Calcule le pourcentage de cellules manquantes sur le data set total.
    Keyword arguments:
    df -- le dataframe

    return : le pourcentage de cellules manquantes de df
    '''
    return df.isna().sum().sum()/(df.size)


def missing_general(df):
    '''Donne un aperçu général du nombre de données manquantes dans le data frame.
    Keyword arguments:
    df -- le dataframe
    '''
    print('Nombre total de cellules manquantes :', missing_cells(df))
    print('Nombre de cellules manquantes en % : {:.2%}'
          .format(missing_cells_perc(df)))


def valeurs_manquantes(df):
    '''Prend un data frame en entrée et créer en sortie un dataframe contenant
    le nombre de valeurs manquantes et leur pourcentage pour chaque variables.
    Keyword arguments:
    df -- le dataframe

    return : dataframe contenant le nombre de valeurs manquantes et
    leur pourcentage pour chaque variable
    '''
    tab_missing = pd.DataFrame(columns=['Variable',
                                        'Missing values',
                                        'Missing (%)'])
    tab_missing['Variable'] = df.columns
    missing_val = list()
    missing_perc = list()

    for var in df.columns:
        nb_miss = missing_cells(df[var])
        missing_val.append(nb_miss)
        perc_miss = missing_cells_perc(df[var])
        missing_perc.append(perc_miss)

    tab_missing['Missing values'] = list(missing_val)
    tab_missing['Missing (%)'] = list(missing_perc)
    return tab_missing


def bar_missing(df):
    '''Affiche le barplot présentant le nombre de données présentes par variable.
    Keyword arguments:
    df -- le dataframe
    '''
    msno.bar(df)
    plt.title('Nombre de données présentes par variable', size=15)
    plt.show()


def barplot_missing(df):
    '''Affiche le barplot présentant le pourcentage de
    données manquantes par variable.
    Keyword arguments:
    df -- le dataframe
    '''
    proportion_nan = df.isna().sum()\
        .divide(df.shape[0]/100).sort_values(ascending=False)

    sns.set(style="whitegrid")
    plt.figure(figsize=(10, 30))
    sns.barplot(y=proportion_nan.index, x=proportion_nan.values)
    plt.title('Pourcentage de données manquantes par variable', size=15)
    plt.show()


def drop_lignes(df, index):
    '''Supprime les lignes des index donnés en argument.
    Keyword arguments:
    df -- le dataframe
    index -- les index des lignes qu'on souhaite supprimer.
    '''
    df.drop(index, axis=0, inplace=True, errors='ignore')
    print('Suppression effectuée')


def df_agg_cust(df):
    '''Prend un data frame en entrée et retourne un dataframe agrégé et
    groupé en fonction de l'identifiant client unique.
    Keyword arguments:
    df -- le dataframe

    return : dataframe agrégé et groupé en fonction de
    l'identifiant client unique
    '''

    date_max = df['order_purchase_timestamp'].max()
    
    data_cust = df.groupby('customer_unique_id').agg(
        order_purchase_timestamp=('order_purchase_timestamp', np.max),
        order_delivered_customer_date=('order_delivered_customer_date', np.max),
        order_estimated_delivery_date=('order_estimated_delivery_date', np.max),
        time_since_first_order=('order_purchase_timestamp', lambda x: (date_max - np.min(x)).days),
        time_since_last_order=('order_purchase_timestamp', lambda x: (date_max - np.max(x)).days),
        customer_city=('customer_city', lambda x: x.mode()[0]),
        customer_state=('customer_state', lambda x: x.mode()[0]),
        nb_total_order=('order_id', 'count'),
        nb_total_item=('product_id', 'count'),
        total_price=('price', np.sum),
        mean_price=('price', np.mean),
        payment_type=('payment_type', lambda x: x.mode()[0]),
        mean_review_score=('review_score', np.mean),
        seller_city=('seller_city', lambda x: x.mode()[0]),
        seller_state=('seller_state', lambda x: x.mode()[0]),
        cat=('product_category_name_english', lambda x: x.mode()[0])
    )

    return data_cust



def plot_multiple_boxplots(df, n_cols=3, figsize=(15,10)):
    """
    Affiche les boxplots de toutes les colonnes quantitatives du DataFrame.

    Arguments :
    df       -- le DataFrame contenant les données
    n_cols   -- nombre de colonnes dans la grille de sous-graphes (par défaut 3)
    figsize  -- taille de la figure matplotlib (largeur, hauteur)

    Affiche un boxplot par variable quantitative.
    """

    # Sélection des colonnes numériques uniquement (int et float)
    quant_cols = df.select_dtypes(include=['int64', 'float64']).columns

    n_vars = len(quant_cols)
    n_rows = math.ceil(n_vars / n_cols)

    fig, axs = plt.subplots(n_rows, n_cols, figsize=figsize)
    axs = axs.flatten()

    for i, col in enumerate(quant_cols):
        sns.boxplot(x=df[col], ax=axs[i], color='skyblue')
        axs[i].set_title(f'Boxplot de {col}', fontsize=12)
        axs[i].set_xlabel('')
        axs[i].set_ylabel('')
        axs[i].tick_params(axis='x', rotation=45)

    # Supprimer les axes inutilisés si il y en a
    for j in range(i+1, len(axs)):
        fig.delaxes(axs[j])

    fig.suptitle('Boxplots des variables quantitatives', fontsize=16)
    fig.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()


def distribution_barplots(df, colonnes, nrows=None, ncols=None, bins=30, kde=True, figsize=None, title=None):
    """
    Affiche les histogrammes pour chaque variable quantitative renseignée.
    
    Arguments:
    df -- dataframe contenant les données
    colonnes -- liste des variables à afficher
    nrows -- nombre de lignes dans la grille de graphiques (optionnel)
    ncols -- nombre de colonnes dans la grille de graphiques (optionnel)
    bins -- nombre de bins pour l'histogramme (par défaut 30)
    kde -- booléen, afficher la courbe KDE (par défaut True)
    figsize -- taille de la figure (tuple) (optionnel)
    title -- titre global de la figure (optionnel)
    """
    # Vérifier que toutes les colonnes sont présentes dans le dataframe
    colonnes_valides = [col for col in colonnes if col in df.columns]
    colonnes_invalides = set(colonnes) - set(colonnes_valides)
    if colonnes_invalides:
        print(f"Attention : colonnes non trouvées dans le dataframe et ignorées : {colonnes_invalides}")

    n = len(colonnes_valides)
    if n == 0:
        print("Aucune colonne valide à afficher.")
        return

    # Calcul automatique du nombre de lignes et colonnes si non spécifié
    if nrows is None or ncols is None:
        ncols = int(n**0.5)
        nrows = (n + ncols - 1) // ncols  # plafond de n/ncols

    # Taille par défaut si pas fourni (ajusté pour nrows/ncols)
    if figsize is None:
        figsize = (4 * ncols, 4 * nrows)

    fig, axs = plt.subplots(nrows, ncols, figsize=figsize)
    axs = axs.flatten() if n > 1 else [axs]

    for i, col in enumerate(colonnes_valides):
        sns.histplot(data=df, x=col, bins=bins, kde=kde, ax=axs[i], color='steelblue')
        axs[i].set_title(col, fontsize=12, fontweight='bold')
        axs[i].set_xlabel('')
        axs[i].set_ylabel('')

    # Supprimer les axes inutilisés
    for j in range(n, nrows * ncols):
        fig.delaxes(axs[j])

    # Titre global
    if title is None:
        title = "Distribution des variables quantitatives"
    fig.suptitle(title, fontsize=16, fontweight='bold')

    plt.tight_layout(rect=[0, 0, 1, 0.96])  # laisser de la place pour le titre
    plt.show()


def test_normalite(df, colonnes, level=0.05):
    """
    Applique plusieurs tests de normalité (Shapiro, D’Agostino, Anderson-Darling) sur les colonnes d’un DataFrame.
    
    Paramètres :
    df : pd.DataFrame — DataFrame contenant les données.
    colonnes : list — Liste des colonnes quantitatives à tester.
    level : float — Niveau de signification (par défaut 0.05).

    Retour :
    DataFrame avec les résultats des tests de normalité.
    """
    resultats = []

    for col in colonnes:
        if df[col].isnull().sum() > 0:
            serie = df[col].dropna()
        else:
            serie = df[col]

        res = {"Variable": col}

        # Test de Shapiro-Wilk (limité à N <= 5000 pour précision)
        if len(serie) <= 5000:
            stat_shapiro, p_shapiro = shapiro(serie)
        else:
            stat_shapiro, p_shapiro = None, None

        # Test de D’Agostino et Pearson
        stat_normaltest, p_normaltest = normaltest(serie)

        # Test d'Anderson-Darling (pas de p-valeur)
        result_anderson = anderson(serie)
        stat_anderson = result_anderson.statistic
        critical_values = result_anderson.critical_values
        significance_level = result_anderson.significance_level

        seuil_index = next((i for i, s in enumerate(significance_level) if s == level*100), None)

        res.update({
            "Shapiro p-val": p_shapiro if p_shapiro is not None else "N > 5000",
            "Normaltest p-val": p_normaltest,
            "Anderson stat": stat_anderson,
            "Anderson seuil 5%": critical_values[seuil_index] if seuil_index is not None else None,
            "Normalité (Shapiro)": p_shapiro > level if p_shapiro is not None else "N/A",
            "Normalité (Normaltest)": p_normaltest > level,
            "Normalité (Anderson)": stat_anderson < critical_values[seuil_index] if seuil_index is not None else "N/A"
        })

        resultats.append(res)

    return pd.DataFrame(resultats)


def plot_bar_count(data, column_name, top_n=None, figsize=(10,5), palette="Blues"):
    if column_name not in data.columns:
        raise ValueError(f"La colonne '{column_name}' n'existe pas dans le DataFrame.")
    
    plt.figure(figsize=figsize)
    value_counts = data[column_name].value_counts()

    if top_n is not None:
        value_counts = value_counts.head(top_n)
    
    cmap = plt.get_cmap(palette)
    colors = [cmap(i / len(value_counts)) for i in range(len(value_counts))]

    ax = sns.barplot(x=value_counts.index, y=value_counts.values, palette=colors)

    ax.set_title(f"Nombre d’occurrences par catégorie - {column_name}", fontsize=14)
    ax.set_xlabel(column_name)
    ax.set_ylabel("Fréquence")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    return ax


def pie_plot(df, colonnes):
    """
    Affiche un camembert pour chaque colonne catégorielle passée en argument,
    avec des couleurs douces dans les nuances de bleu et violet pour chaque catégorie,
    et des pourcentages petits et lisibles.
    
    Arguments :
    df -- DataFrame contenant les données
    colonnes -- liste des colonnes à afficher
    """
    n = len(colonnes)
    fig, axs = plt.subplots(1, n, figsize=(6*n, 6))
    if n == 1:
        axs = [axs]

    for i, col in enumerate(colonnes):
        counts = df[col].value_counts(dropna=False)
        labels = counts.index.astype(str)
        sizes = counts.values
        
        # Palette douce bleu/violet avec autant de couleurs que de catégories
        palette = sns.color_palette("cool", len(labels))
        
        axs[i].pie(
            sizes, 
            labels=labels, 
            colors=palette, 
            autopct='%1.1f%%', 
            startangle=140, 
            textprops={'fontsize': 9, 'color': 'dimgray'}
        )
        axs[i].set_title(f'Répartition de {col}', fontsize=14)
    
    plt.tight_layout()
    plt.show()



def boxplot_relation(df, colonnes, var_comparaison, longueur,
                     largeur, ordre=None, outliers=True, option=False):
    '''Affiche les boxplot des colonnes en fonctions de var_comparaison.
    Keyword arguments:
    df -- le dataframe
    colonnes -- variables à afficher
    var_comparaison -- variable avec laquelle comparer
    longueur -- nombre de figure en longueur
    largeur -- nombre de figure en largeur
    ordre -- ordre dans lequel placer les valeurs catégorielles (default None)
    outliers -- afficher ou non les valeurs jugées
    comme outliers par le boxplot (default True)
    option -- afficher les labels de x avec une rotation de 90° (default False)
    '''
    fig = plt.figure(figsize=(40, 60))
    for i, col in enumerate(colonnes, 1):
        ax = fig.add_subplot(longueur, largeur, i)
        sns.boxplot(x=df[var_comparaison], y=df[col],
                    ax=ax, order=ordre, showfliers=outliers)
        if option:
            plt.xticks(rotation=90, ha='right')
    fig.suptitle('Boxplot de chaque target en fonction de {}'
                 .format(var_comparaison))
    plt.tight_layout(pad=4)
    plt.show()
